# <font color='palegoldenrod'>Final project - sentiment analysis comparison part 3</font>
using hagging face wrapper and pytorch

#Imports & functions

In [ ]:
import pandas as pd
import numpy as np
import math
import string
#connect to drive
from google.colab import drive
# visualisation
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
!pip install -qq transformers
from transformers import pipeline, DistilBertTokenizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 32.7 MB/s eta 0:00:00


In [ ]:
#accuracy precision recall
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# Load data

In [ ]:
# data is from : https://www.kaggle.com/datasets/therohk/ireland-historical-news?datasetId=30661
drive.mount('/content/drive')
path = "/content/drive/MyDrive/final project/data_files/nlp_data.csv"
df = pd.read_csv(path)
df.drop(['Unnamed: 0'], axis='columns', inplace=True)
print(df.shape)
df.head()

Mounted at /content/drive
(50000, 3)


,label,clean_reviews,final_tokenized
0,0.0,thats keep ask many fight scream match swear g...,"['thats', 'keep', 'ask', 'many', 'fight', 'scr..."
1,0.0,watch entire movie could watch entire movie st...,"['watch', 'entire', 'movie', 'could', 'watch',..."
2,1.0,touch love story reminiscent mood love draw he...,"['touch', 'love', 'story', 'reminiscent', 'moo..."
3,0.0,latterday Fulci schlocker totally abysmal conc...,"['latterday', 'Fulci', 'schlocker', 'totally',..."
4,0.0,First firmly believe Norwegian movie continual...,"['First', 'firmly', 'believe', 'Norwegian', 'm..."


## split to train and test

In [ ]:
# 70% for training, 15% for validation and 15% for testing
# df_train, df_test = train_test_split(df,test_size=0.15, random_state = 0)
# keeping the split of the train-test same as the fine-tuned model
df_train,df_val_temp=train_test_split(df, test_size=0.3, random_state = 0)
df_val,df_test=train_test_split(df_val_temp, test_size=0.5, random_state = 0)

print(df_train.shape, df_test.shape)

(35000, 3) (7500, 3)


# Predict

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", tokenizer=tokenizer,truncation=True,max_length=512)

# Apply sentiment analysis on reviews
df_test['pred'] = df_test['clean_reviews'].apply(lambda x: sentiment_pipeline(x)[0]['label'])

In [ ]:
df_test.head()

,label,clean_reviews,final_tokenized,pred
48314,1.0,collect film super8 manage snag full length pr...,"['collect', 'film', 'super8', 'manage', 'snag'...",POSITIVE
13439,0.0,although lead actress STRIKINGLY beautiful plo...,"['although', 'lead', 'actress', 'STRIKINGLY', ...",NEGATIVE
20439,0.0,jon good wife simply one bad title film ever r...,"['jon', 'good', 'wife', 'simply', 'one', 'bad'...",NEGATIVE
41199,0.0,star Eugene morris jerome high school adaptati...,"['star', 'Eugene', 'morris', 'jerome', 'high',...",NEGATIVE
18272,0.0,gregory peck act excellent one would expect ci...,"['gregory', 'peck', 'act', 'excellent', 'one',...",NEGATIVE


In [ ]:
df_test['pred_num']=0
df_test['pred_num'].loc[df_test['pred']!='NEGATIVE']=1

<ipython-input-11-8b9c3185c1a0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['pred_num'].loc[df_test['pred']!='NEGATIVE']=1


# Results

In [ ]:
#get scores
print("accuracy score :",accuracy_score(df_test['label'], df_test['pred_num']))
print("recall score :",recall_score(df_test['label'], df_test['pred_num']))
print("precision score :",precision_score(df_test['label'], df_test['pred_num']))

accuracy score : 0.7865333333333333
recall score : 0.6330915928015042
precision score : 0.9093364197530864
